# Разработка чат-бота

Раздел содержит пример разработки чат-бота на основе LLM.
Этот чат-бот сможет вести беседу и запоминать предыдущие действия пользователя.

В примере рассмотрен чат-бот, который для ведения беседы использует только языковую модель.
Также существуют сопособы создания чат-ботов, которые могут вас заинтересовать:

- [Conversational RAG](/docs/tutorials/qa_chat_history) — позволяет чат-боту использовать внешние источники данных.
- [Агенты](/docs/tutorials/agents) — чат-боты, которые может выполнять действия.

В этом разделе вы найдете базовую информацию о разработке чат-ботов, которая будет полезна при работе с приведенными выше разделами.
Но если нужно вы можете сразу начать с более сложных чат-ботов.

## Основные понятия

Основные компоненты, с которыми вы будете работать:

- [`Чат-модели`](/docs/concepts/#chat-models). Чат-боты работают с сообщениями, а не необработанным текстом. Поэтому для разработки лучше подходят для чат-модели, а не текстовые LLM.
- [`Шаблоны промптов`](/docs/concepts/#prompt-templates), которые упрощают процесс создания промптов, объединяющих стандартные сообщения, ввод пользователя, историю чатов и, при необходимости, дополнительный извлеченный контекст.
- [`История чата`](/docs/concepts/#chat-history), которая позволяет чат-боту "запоминать" прошлые взаимодействия и учитывать их при ответе на последующие вопросы.

<!--
- Отладка и трассировка вашего приложения с помощью [LangSmith](/docs/concepts/#langsmith)
-->

На наглядном примере вы узнаете, как объединить вышеупомянутые компоненты для создания мощного развитого чат-бота.

## Подготовка к разработке

### Jupyter-блокноты

Это руководство, как и большинство других в документации, использует [Jupyter-блокноты](https://jupyter.org/). Они отлично подходят для изучения работы с LLM-системами, так как предоставляют интерактивную среду для работы с руководствами и позволяют работать с непредвиденными ситуациями: недоступностью API, нетипичным выводом и другими.

Подробнее об установке jupyter — в [официальной документации](https://jupyter.org/install).

### Установка

Для установки GigaChain выполните команды:

```{=mdx}
import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
    <CodeBlock language="bash">pip install langchain</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
    <CodeBlock language="bash">conda install langchain -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>
```


Подробнее об установке — в разделе [Установка](https://developers.sber.ru/docs/ru/gigachain/get-started/installation).

<!--
### LangSmith

Многие приложения, которые вы создаете с помощью LangChain, будут содержать несколько шагов с многократными вызовами LLM.
По мере усложнения этих приложений становится важно иметь возможность инспектировать, что именно происходит внутри вашей цепочки или агента.
Лучший способ сделать это — с помощью [LangSmith](https://smith.langchain.com).

После регистрации по ссылке выше, убедитесь, что вы установили переменные среды для начала ведения журнала трассировок:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

Или, если вы работаете в ноутбуке, вы можете установить их с помощью:

```python
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```
-->

## Быстрый старт

Сначала ознакомьтесь как использовать языковую модель отдельно.
GigaChain поддерживает различные языковые модели, которые могут заменять друг друга.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-3.5-turbo"`} />
```

In [1]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

Сначала попробуйте использовать модель напрямую.
`ChatModel` — это экземпляры Runnable-интерфейса GigaChain, что означает, что для работы они они предоставляют стандартный интерфейс.
Для простого вызова модели, вы можете передать список сообщений в метод `.invoke`.

In [2]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-be38de4a-ccef-4a48-bf82-4292510a8cbf-0')

Сама по себе модель не имеет понятия состояния.
Это можно увидеть, если задать модели дополнительный вопрос:

In [3]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, as an AI assistant, I do not have the capability to know your name unless you provide it to me.", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 12, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_caf95bb1ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-8d8a9d8b-dddb-48f1-b0ed-ce80ce5397d8-0')

<!--
Давайте взглянем на пример [трассировки LangSmith](https://smith.langchain.com/public/5c21cb92-2814-4119-bae9-d02b8db577ac/r)

Мы видим, что модель не учитывает предыдущий ход разговора и не может ответить на вопрос. Это делает чат-бот крайне неудобным!
-->

Чтобы обойти это ограничение, передайте всю историю разговора в модель:

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 35, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-5692718a-5d29-4f84-bad1-a9819a6118f1-0')

Качество ответа модели заметно возросло.

Работа с историей сообщений — это техника, которая лежит в основе способности чат-бота вести разговор.
Ниже вы узнаете как лучше ее реализовать.

## История сообщений

Чтобы модель сохраняла состояние, вы можете обернуть ее в класс Message History.
Класс будет отслеживать входные и выходные данные модели и сохранять их в хранилище данных.
При повторных обращениях сообщения модели будут загружаться из хранилища и передаваться в цепочку в качестве части входных данных.

Пример ниже использует хранилище истории сообщений, доступное в пакете `gigachain-community`.
Убедитесь, что установили его:

In [5]:
# ! pip install gigachain_community

После этого вы можем импортировать соответствующие классы и настроить цепочку, которая обернет модель и добавит историю сообщений.
Самой важной частью здесь является функция, которую мы передаем в качестве `get_session_history`.
Эта функция должна принимать `session_id` и возвращать объект Message History.
Параметр `session_id` используется для различения отдельных разговоров и передается как часть конфигурационной переменной при вызове новой цепочки.

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

Создайте конфигруационную переменную `config`, которая будет содержать дополнительные данные, полезные для вызова цепочки.
В приведенном примере вам нужно передавать в этой переменной `session_id`.
Переменную нужно передавать при каждом вызове runnable.

In [14]:
conf = {"configurable": {"session_id": "abc2"}}

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=conf,
)

response.content

'Hello Bob! How can I assist you today?'

In [16]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=conf,
)

response.content

'Your name is Bob.'

Теперь ваш чат-бот запоминает информацию.
Если вы измените переменную config, чтобы сослаться на другой `session_id`, то увидите, что разговор начнется заново.

In [17]:
conf = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=conf,
)

response.content

"I'm sorry, I do not have the ability to know your name unless you tell me."

При этом вы всегда можете вернуться к первоначальному разговору (так как он сохраняется его в базе данных).

In [18]:
conf = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=conf,
)

response.content

'Your name is Bob.'

Таким образом, ваш чат-бот сможет вести беседы с множеством пользователей.

Ниже вы узнаете как развить и персонализировать данные, которые сохраняет чат-бот, с помощью шаблона промпта.

## Шаблоны промптов

Шаблоны промптов помогают преобразовать необработанную информацию от пользователя в формат, с которым может работать LLM.
В данном случае необработанный ввод представляет собой сообщение, которое передается в LLM.
Это сообщение можно усложнить.
Сначала добавьте в него системное сообщение с набором собвственных инструкций (но все еще принимая сообщения в качестве ввода).
Затем дополните сообщения вспомогательными входными данными.

Для добавления системного сообщения создайте экземпляр `ChatPromptTemplate`.
Чтобы передать все сообщения используйте `MessagesPlaceholder`.

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

В результате тип входных данных изменится — вместо передачи списка сообщений вы теперь передаете словарь с ключом `messages`, который содержит список сообщений.

In [21]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

'Hello, Bob! How can I assist you today?'

Теперь вы можете обернуть полученный код в объект истории сообщений `with_message_history`, созданный ранее.

In [22]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [23]:
conf = {"configurable": {"session_id": "abc5"}}

In [24]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=conf,
)

response.content

'Hello, Jim! How can I assist you today?'

In [25]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=conf,
)

response.content

'Your name is Jim. How can I assist you further, Jim?'

Усложните полученный промпт.
Предположим, что шаблон промпта теперь выглядит примерно так:

In [27]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

В примере выше в промпт добавлена новая переменная `language`.
Теперь вы можем вызвать цепочку и передать язык на свой выбор выбор.

In [28]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola Bob! ¿En qué puedo ayudarte hoy?'

Оберните полученную цепочку в класс `with_message_history`.
Теперь, поскольку входные данные содержать несколько ключей, вам нужно указать правильный ключ для сохранения истории чата.

In [29]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [33]:
conf = {"configurable": {"session_id": "abc11"}}

In [34]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=conf,
)

response.content

'¡Hola Todd! ¿En qué puedo ayudarte hoy?'

In [35]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=conf,
)

response.content

'Tu nombre es Todd. ¿Hay algo más en lo que pueda ayudarte?'

<!--
Чтобы лучше понять, что происходит внутри, ознакомьтесь с [этой трассировкой LangSmith](https://smith.langchain.com/public/f48fabb6-6502-43ec-8242-afc352b769ed/r).
-->

## Управление историей разговоров

При разработке чат-бота рано или поздно вам понадобится управлять историей разговоров.
Это связанно с тем, что в определенный момент количество и содержимое сообщений превзойдут размер контекстного окна LLM.
Поэтому вам нужно добавить этап, на котором будет ограничиваться размер передваваемых сообщений.

:::caution

При этом, этот этап должен срабатывать до шаблона промпта, но после загрузки пердыдущих сообщений из Message History.

:::

Для этого перед промптом вы можете добавить простой шаг, который изменяет ключ `messages` соотетствующим образом, а затем обернуть полученную цепочку в класс Message History.
Сначала определите функцию, которая будет изменять передаваемые сообщения.
Пусть она выбирает `k` самых последних сообщений.
Затем вы можете создать новую цепочку, добавив их в начало.

In [42]:
from langchain_core.runnables import RunnablePassthrough


def filter_messages(messages, k=10):
    return messages[-k:]


chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

Теперь если вы создадите список сообщений длиной более 10 сообщений, вы увидите, что модель больше не запоминает информацию из первых сообщений.

In [43]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [47]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, I don’t have access to your name. Can I help you with anything else?"

Но если вы спросите информацию, которая находится в последних десяти сообщениях, модель покажет, что все еще ее помнит.

In [48]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
        "language": "English",
    }
)
response.content

'You mentioned that you like vanilla ice cream.'

Теперь оберните полученный код в Message History.

In [55]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

conf = {"configurable": {"session_id": "abc20"}}

In [57]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=conf,
)

response.content

"I'm sorry, I don't know your name."

Теперь история чата содержит два новых сообщения.
Это значит, что еще больше информации, которая ранее хранилась истории разговоров, теперь недоступна.

In [58]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="whats my favorite ice cream?")],
        "language": "English",
    },
    config=conf,
)

response.content

"I'm sorry, I don't know your favorite ice cream flavor."

<!--
Если вы посмотрите на LangSmith, вы сможете увидеть, что происходит под капотом, в [трассировке LangSmith](https://smith.langchain.com/public/fa6b00da-bcd8-4c1c-a799-6b32a3d62964/r).
-->

## Потоковая передача

Потоковая передача — одна из важных составляющих пользовательского опыта для чат-ботов.
Большие языковые модели могут долго отвечать, поэтому для повышения отзывчивости большинство приложений обрабатывает и отображает каждый токен по мере его генерации.
Это позволяет пользователю видеть прогресс.

Для работы с потоковой передачей все цепочки предоставляют метод `.stream`, и те, что используют историю сообщений, не исключение.
Используйте этот метод, чтобы получить потоковый ответ.

In [40]:
conf = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=conf,
):
    print(r.content, end="|")

|Sure|,| Todd|!| Here|'s| a| joke| for| you|:

|Why| don|'t| scientists| trust| atoms|?

|Because| they| make| up| everything|!||

## Смотрите также

- [Conversational RAG](/docs/tutorials/qa_chat_history) — позволяет чат-боту использовать внешние источники данных.
- [Агенты](/docs/tutorials/agents) — чат-боты, которые может выполнять действия.
- [Работа с потоковой передача](/docs/how_to/streaming) — потоковая передача очень важна для чат-приложений.
- [Работа с историей сообщений](/docs/how_to/message_history) — раздел с подробной информацией о работе с историей сообщений.